In [3]:
import os
import glob
import pandas as pd
from pathlib import Path
import re
from pandas.errors import EmptyDataError
import numpy as np

# Analysis of changes from 2024/25 to 2025/26 season

In [4]:
# --- Paths
PATH_2425 = Path(r"C:\Users\bpali\PycharmProjects\SquashApp\previous_seasons\2024-2025\players_df")
PATH_2526 = Path(r"C:\Users\bpali\PycharmProjects\SquashApp\2025-2026\players_df\week_0")

In [5]:
def read_csv_robust(p: Path) -> pd.DataFrame | None:
    """Return a DataFrame or None if the file is empty/unreadable."""
    try:
        if p.stat().st_size == 0:
            return None
    except FileNotFoundError:
        return None

    for enc in ("utf-8-sig", "utf-8", "cp1252", "latin1"):
        try:
            df = pd.read_csv(p, encoding=enc)
            # Guard against files that have no columns after parsing
            if getattr(df, "empty", False) and len(df.columns) == 0:
                return None
            return df
        except EmptyDataError:
            return None
        except Exception:
            # try next encoding
            continue
    # final attempt without encoding hint
    try:
        df = pd.read_csv(p)
        if getattr(df, "empty", False) and len(df.columns) == 0:
            return None
        return df
    except Exception:
        return None

In [6]:
# --- Parse division key from filename:
# Accepts "11_players_df.csv", "Premier Main_players_df.csv", "7A_players_df.csv", etc.
DIV_SUFFIX_RE = re.compile(r"_players_df(?:\.csv)?$", re.IGNORECASE)

In [7]:
def division_key_from_filename(filename: str) -> str:
    base = Path(filename).name
    base = DIV_SUFFIX_RE.sub("", base)
    return base.strip()

In [8]:
# --- week_X extractor
WEEK_RE = re.compile(r"week_(\d+)$", re.IGNORECASE)

def week_num_from_dir(p: Path) -> int | None:
    m = WEEK_RE.search(p.name)
    return int(m.group(1)) if m else None

## 2024/25: pick the latest available players_df per division across week folders

In [9]:
def collect_latest_players_2425(root_players_df: Path) -> dict[str, dict]:
    """
    Walk week_* folders from highest to lowest and keep the first *non-empty* CSV for each division key.
    Returns: {division_key: {"path": Path, "week": int}}
    """
    assert root_players_df.exists(), f"Not found: {root_players_df}"
    week_dirs = [p for p in root_players_df.glob("week_*") if p.is_dir()]
    week_dirs = sorted(week_dirs, key=lambda p: week_num_from_dir(p) or -1, reverse=True)

    chosen: dict[str, dict] = {}
    skipped_empty: list[tuple[str, int, str]] = []

    for wdir in week_dirs:
        w = week_num_from_dir(wdir)
        if w is None:
            continue
        for csv_path in wdir.glob("*.csv"):
            div_key = division_key_from_filename(csv_path.name)
            if not div_key or div_key in chosen:
                continue
            # quick zero-byte check
            if csv_path.stat().st_size == 0:
                skipped_empty.append((div_key, w, str(csv_path)))
                continue
            # peek to ensure it actually parses
            df = read_csv_robust(csv_path)
            if df is None or (len(df.columns) == 0):
                skipped_empty.append((div_key, w, str(csv_path)))
                continue
            # Accept this file for the division
            chosen[div_key] = {"path": csv_path, "week": w}

    # Report any empties encountered (optional)
    if skipped_empty:
        print(f"Skipped {len(skipped_empty)} empty/unreadable CSV(s) while scanning 2024/25 (showing up to 10):")
        for div_key, w, p in skipped_empty[:10]:
            print(f"  - week_{w} {div_key}: {p}")

    return chosen

latest_2425 = collect_latest_players_2425(PATH_2425)
print(f"2024/25 divisions captured: {len(latest_2425)}")


Skipped 1 empty/unreadable CSV(s) while scanning 2024/25 (showing up to 10):
  - week_21 15B: C:\Users\bpali\PycharmProjects\SquashApp\previous_seasons\2024-2025\players_df\week_21\15B_players_df.csv
2024/25 divisions captured: 27


## 3) Load 2024/25 DataFrames and combine

In [10]:
def load_players_season(chosen_map: dict[str, dict], season_label: str):
    frames = []
    skipped = []
    for div_key, meta in chosen_map.items():
        df = read_csv_robust(meta["path"])
        if df is None or len(df.columns) == 0:
            skipped.append((div_key, meta["week"], str(meta["path"])))
            continue
        df = df.copy()
        df["season"] = season_label
        df["division_key"] = div_key
        df["source_week"] = meta["week"]
        frames.append(df)
    combo = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
    if skipped:
        print(f"Skipped {len(skipped)} file(s) for {season_label} due to empty/unreadable content (showing up to 10):")
        for div_key, w, p in skipped[:10]:
            print(f"  - {div_key} (week_{w}): {p}")
    return combo

players_2425 = load_players_season(latest_2425, "2024-2025")
print(players_2425.shape)
players_2425.head()


(2020, 9)


,Order,Player,HKS No.,Ranking,Points,Team,season,division_key,source_week
0,1,Fu Ivan Fuk Cheong,27126,31,15.00,7NT,2024-2025,11,22
1,2,Ko Sze Chai Cedric,32055,16,18.50,7NT,2024-2025,11,22
2,3,Hui Adrian Wai Hung,25636,26,15.50,7NT,2024-2025,11,22
3,4,Chan Raymond Tsan Fai,23067,35,14.54,7NT,2024-2025,11,22
4,5,Yung Kai Sing,32148,61,10.07,7NT,2024-2025,11,22


In [11]:
# --- Manual override(s) for specific divisions → specific weeks
def apply_manual_override(chosen_map, root_path: Path, division_key: str, week: int):
    cand = root_path / f"week_{week}" / f"{division_key}_players_df.csv"
    if not cand.exists():
        raise FileNotFoundError(f"Override file not found: {cand}")
    chosen_map[division_key] = {"path": cand, "week": week}
    print(f"Override applied: {division_key} → week_{week} ({cand})")

# Force 15B to use week_20
apply_manual_override(latest_2425, PATH_2425, "15B", 20)

# sanity check
latest_2425.get("15B")


Override applied: 15B → week_20 (C:\Users\bpali\PycharmProjects\SquashApp\previous_seasons\2024-2025\players_df\week_20\15B_players_df.csv)


{'path': WindowsPath('C:/Users/bpali/PycharmProjects/SquashApp/previous_seasons/2024-2025/players_df/week_20/15B_players_df.csv'),
 'week': 20}

## 4) Load 2025/26 week_0 with the same resilience

In [12]:
def load_all_players_from_folder(folder: Path, season_label: str) -> pd.DataFrame:
    assert folder.exists(), f"Not found: {folder}"
    frames, skipped = [], []
    for csv_path in folder.glob("*.csv"):
        div_key = division_key_from_filename(csv_path.name)
        df = read_csv_robust(csv_path)
        if df is None or len(df.columns) == 0:
            skipped.append((div_key, 0, str(csv_path)))
            continue
        df = df.copy()
        df["season"] = season_label
        df["division_key"] = div_key
        df["source_week"] = 0
        frames.append(df)
    if skipped:
        print(f"Skipped {len(skipped)} file(s) for {season_label} due to empty/unreadable content (showing up to 10):")
        for div_key, w, p in skipped[:10]:
            print(f"  - {div_key} (week_{w}): {p}")
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

players_2526 = load_all_players_from_folder(PATH_2526, "2025-2026")
print(players_2526.shape)
players_2526.head()


(2066, 9)


,Order,Player,HKS No.,Ranking,Points,Team,season,division_key,source_week
0,1,Foo Chun Huey,32317,0,0.0,Banbil,2025-2026,10,0
1,2,Au Ming Chung Sampson,27153,0,0.0,Banbil,2025-2026,10,0
2,3,Leow Kin Leung,33698,0,0.0,Banbil,2025-2026,10,0
3,4,So Li Chuan John,33576,0,0.0,Banbil,2025-2026,10,0
4,5,Leung Wai Kwan,31027,0,0.0,Banbil,2025-2026,10,0


In [13]:
def division_tier_from_key(s: str):
    """
    Map division_key → integer Main-league tier.
    - 'Premier Main' → 1
    - '7A'/'7B' → 7 (strip trailing A/B)
    - '10', '11', '12', ... → that integer
    - Ignore Ladies/Masters streams entirely → NaN
    Anything unrecognized → NaN (so it’s easy to spot).
    """
    if s is None or (isinstance(s, float) and pd.isna(s)):
        return np.nan
    x = str(s).strip()
    xl = x.lower()

    # Ignore Ladies/Masters (incl. 'Premier Ladies', 'Premier Masters', 'L2', 'M3', etc.)
    if 'ladies' in xl or xl.startswith('l'):
        return np.nan
    if 'masters' in xl or xl.startswith('m'):
        return np.nan

    # Premier Main → 1
    if xl in {'premier main', 'premier-main', 'premier_main', 'premier'}:
        return 1

    # Remove spaces then match N or N + optional trailing A/B
    xs = x.replace(' ', '')
    m = re.fullmatch(r'(\d+)[ABab]?$', xs)
    if m:
        return int(m.group(1))

    # Fallback: any integer found
    m2 = re.search(r'(\d+)', xl)
    if m2:
        return int(m2.group(1))

    return np.nan

def add_division_tier_column(df: pd.DataFrame, src_col='division_key', out_col='division_tier_main'):
    if src_col not in df.columns:
        raise KeyError(f"Column '{src_col}' not found in DataFrame")
    out = df[src_col].apply(division_tier_from_key)
    # nullable integer dtype so NaNs are allowed
    df[out_col] = out.astype('Int64')
    return df

# Apply to both season dataframes
add_division_tier_column(players_2425, 'division_key', 'division_tier_main')
add_division_tier_column(players_2526, 'division_key', 'division_tier_main')

# quick sanity peek
print(sorted(players_2425['division_key'].unique().tolist()))
players_2425[['division_key','division_tier_main']].drop_duplicates().sort_values(['division_tier_main','division_key']).head(30)

['10', '11', '12', '13A', '13B', '14', '15A', '15B', '2', '3', '4', '5', '6', '7A', '7B', '8A', '8B', '9', 'L2', 'L3', 'L4', 'M2', 'M3', 'M4', 'Premier Ladies', 'Premier Main', 'Premier Masters']


,division_key,division_tier_main
2000,Premier Main,1
1416,2,2
1631,3,3
695,4,4
1707,5,5
1797,6,6
773,7A,7
843,7B,7
1893,8A,8
912,8B,8


In [14]:
# Find the Team column (handles slight naming differences)
def find_team_col(df: pd.DataFrame):
    candidates = {'team', 'team_name', 'home_team', 'away_team'}
    lower_map = {c.lower(): c for c in df.columns}
    for key in candidates:
        if key in lower_map:
            return lower_map[key]
    # fallback: anything containing "team"
    for c in df.columns:
        if 'team' in c.lower():
            return c
    raise KeyError("Could not find a Team column. Please set team_col manually.")

def derive_club_from_team(s: str | float) -> str | None:
    """
    Return one of {'HKFC','HKCC','KCC'} or None if not one of the big three.
    Rules (case-insensitive):
      - contains 'Hong Kong Cricket Club' or 'HKCC'  -> HKCC
      - contains 'Kowloon'                            -> KCC
      - contains 'Football'                           -> HKFC
    Priority matters to avoid accidental matches like 'Cricket Club' in KCC names.
    """
    if s is None or (isinstance(s, float) and pd.isna(s)):
        return None
    t = str(s).lower()

    # HKCC (be specific to HKCC/Hong Kong Cricket Club)
    if 'hong kong cricket club' in t or re.search(r'\bhkcc\b', t):
        return 'HKCC'
    # KCC
    if 'kowloon' in t:
        return 'KCC'
    # HKFC
    if 'football' in t:
        return 'HKFC'
    return None

def add_club_column(df: pd.DataFrame, team_col: str | None = None, out_col='Club') -> pd.DataFrame:
    if team_col is None:
        team_col = find_team_col(df)
    df[out_col] = df[team_col].apply(derive_club_from_team)
    return df

# apply to both season dataframes
add_club_column(players_2425)
add_club_column(players_2526)

# Quick check
players_2425[['division_key','Team','Club']].head(10)


,division_key,Team,Club
0,11,7NT,None
1,11,7NT,None
2,11,7NT,None
3,11,7NT,None
4,11,7NT,None
5,11,7NT,None
6,11,7NT,None
7,11,7NT,None
8,11,7NT,None
9,11,7NT,None


### Change in Teams

In [15]:
print(f"Teams in 2024/25: {players_2425['Team'].nunique()}")
print(f"Teams in 2025/26: {players_2526['Team'].nunique()}")

Teams in 2024/25: 215
Teams in 2025/26: 224


### Change in Players

In [16]:
print(f"Players in 2024/25: {players_2425['HKS No.'].nunique()}")
print(f"Players in 2025/26: {players_2526['HKS No.'].nunique()}")

Players in 2024/25: 1715
Players in 2025/26: 1733


In [ ]:
# --- Helper: find the player ID column (defaults to 'HKS No.' but is flexible)
def find_player_id_col(df: pd.DataFrame):
    candidates = {'hks no.', 'hks no', 'hks_no', 'hksno', 'player_id', 'player no', 'player_no'}
    lower_map = {c.lower(): c for c in df.columns}
    for key in candidates:
        if key in lower_map:
            return lower_map[key]
    # last resort: look for a numeric-ish "HKS" column
    for c in df.columns:
        if 'hks' in c.lower():
            return c
    raise KeyError("Could not find an HKS/player ID column. Please set player_id_col manually.")

# --- Stream detector from division_key
def stream_from_division_key(x: str) -> str:
    if x is None:
        return 'Main'
    s = str(x).strip().lower()
    if 'ladies' in s or s.startswith('l'):
        return 'Ladies'
    if 'masters' in s or s.startswith('m'):
        return 'Masters'
    return 'Main'

# --- Compute each player's Main-league tier for the season
# We take the BEST (lowest number) tier the player appears in within Main.
def compute_player_main_tier_map(df: pd.DataFrame, player_id_col: str, main_tier_col='division_tier_main') -> pd.DataFrame:
    main_df = df[df[main_tier_col].notna()].copy()
    if main_df.empty:
        return pd.DataFrame(columns=[player_id_col, 'player_main_tier'])
    agg = (main_df
           .groupby(player_id_col, as_index=False)[main_tier_col]
           .min()
           .rename(columns={main_tier_col: 'player_main_tier'}))
    return agg

# --- Core function: for Masters/Ladies divisions, average the players' Main tier (ignoring those with no Main appearances)
def average_main_tier_for_special_divisions(df: pd.DataFrame,
                                            player_id_col: str | None = None,
                                            division_key_col='division_key',
                                            season_col='season',
                                            main_tier_col='division_tier_main') -> pd.DataFrame:
    if player_id_col is None:
        player_id_col = find_player_id_col(df)

    work = df.copy()
    work['stream'] = work[division_key_col].apply(stream_from_division_key)

    # Only rows from Masters/Ladies streams
    special = work[work['stream'].isin(['Masters', 'Ladies'])].copy()
    if special.empty:
        return pd.DataFrame(columns=[season_col, 'stream', division_key_col, 'n_players', 'avg_main_tier', 'median_main_tier'])

    # Each player's best Main tier in this season
    main_tier_map = compute_player_main_tier_map(work, player_id_col, main_tier_col)

    # Merge, keep only players who actually have a Main tier
    merged = (special
              .merge(main_tier_map, on=player_id_col, how='left')
              .dropna(subset=['player_main_tier']))

    # Optional: dedupe player×division to avoid double-counting the same player twice within the same division
    merged = merged.drop_duplicates(subset=[season_col, division_key_col, player_id_col])

    # Aggregate per (season, stream, division_key)
    out = (merged
           .groupby([season_col, 'stream', division_key_col], as_index=False)
           .agg(n_players=(player_id_col, 'nunique'),
                avg_main_tier=('player_main_tier', 'mean'),
                median_main_tier=('player_main_tier', 'median')))

    # Tidy: sort by stream then by the numeric tier inside the division label where possible
    def numeric_from_label(lbl: str):
        if isinstance(lbl, str):
            if 'premier' in lbl.lower():
                return 1
            m = re.search(r'(\d+)', lbl)
            if m:
                return int(m.group(1))
        return 999
    out['_sort_tier'] = out[division_key_col].apply(numeric_from_label)
    out = out.sort_values([season_col, 'stream', '_sort_tier', division_key_col]).drop(columns=['_sort_tier'])

    # Round the averages for display (keep raw precision if you prefer)
    out['avg_main_tier'] = out['avg_main_tier'].round(2)
    out['median_main_tier'] = out['median_main_tier'].astype('Int64')

    return out


In [18]:
avg_main_by_special_2425 = average_main_tier_for_special_divisions(players_2425)
avg_main_by_special_2526 = average_main_tier_for_special_divisions(players_2526)

display(avg_main_by_special_2425)
display(avg_main_by_special_2526)

,season,stream,division_key,n_players,avg_main_tier,median_main_tier
3,2024-2025,Ladies,Premier Ladies,20,2.75,3
0,2024-2025,Ladies,L2,31,6.0,5
1,2024-2025,Ladies,L3,21,9.48,9
2,2024-2025,Ladies,L4,38,12.74,13
7,2024-2025,Masters,Premier Masters,25,3.52,3
4,2024-2025,Masters,M2,39,6.03,6
5,2024-2025,Masters,M3,60,8.17,7
6,2024-2025,Masters,M4,69,10.61,10


,season,stream,division_key,n_players,avg_main_tier,median_main_tier
3,2025-2026,Ladies,Premier Ladies,16,2.62,2
0,2025-2026,Ladies,L2,33,5.73,6
1,2025-2026,Ladies,L3,23,8.83,8
2,2025-2026,Ladies,L4,49,12.59,13
7,2025-2026,Masters,Premier Masters,29,3.69,4
4,2025-2026,Masters,M2,41,5.71,6
5,2025-2026,Masters,M3,49,7.76,7
6,2025-2026,Masters,M4,91,10.48,10


In [36]:
# count n_players in stream Ladies in avg_main_by_special_2425
n_players_ladies_2425 = avg_main_by_special_2425[avg_main_by_special_2425["stream"] == "Ladies"]["n_players"].sum()
print(f"Total Ladies players in 2024/25: {n_players_ladies_2425}")
# count n_players in stream Ladies in avg_main_by_special_2526
n_players_ladies_2526 = avg_main_by_special_2526[avg_main_by_special_2526["stream"] == "Ladies"]["n_players"].sum()
print(f"Total Ladies players in 2025/26: {n_players_ladies_2526}")

Total Ladies players in 2024/25: 110
Total Ladies players in 2025/26: 121


In [37]:
# count n_players in stream Ladies in avg_main_by_special_2425
n_players_masters_2425 = avg_main_by_special_2425[avg_main_by_special_2425["stream"] == "Masters"]["n_players"].sum()
print(f"Total Ladies players in 2024/25: {n_players_masters_2425}")
# count n_players in stream Ladies in avg_main_by_special_2526
n_players_masters_2526 = avg_main_by_special_2526[avg_main_by_special_2526["stream"] == "Masters"]["n_players"].sum()
print(f"Total Ladies players in 2025/26: {n_players_masters_2526}")

Total Ladies players in 2024/25: 193
Total Ladies players in 2025/26: 210


In [19]:
players_2526[players_2526["Team"].str.contains("JESSICA")]["Team"].unique()

array(['JESSICA L2', 'JESSICA L3', 'JESSICA Mochis', 'JESSICA L4',
       'JESSICA L1'], dtype=object)

In [20]:
# Reuse helpers from earlier cells:
# - find_player_id_col
# - stream_from_division_key
# - compute_player_main_tier_map

def average_main_tier_for_special_divisions_by_club(df: pd.DataFrame,
                                                    player_id_col: str | None = None,
                                                    division_key_col='division_key',
                                                    club_col='Club',
                                                    season_col='season',
                                                    main_tier_col='division_tier_main',
                                                    clubs_keep=('HKFC','HKCC','KCC')) -> pd.DataFrame:
    if player_id_col is None:
        player_id_col = find_player_id_col(df)

    work = df.copy()
    work['stream'] = work[division_key_col].apply(stream_from_division_key)

    # Masters & Ladies rows only, and restrict to the 3 clubs of interest
    special = work[work['stream'].isin(['Masters', 'Ladies'])].copy()
    special = special[special[club_col].isin(clubs_keep)]
    if special.empty:
        return pd.DataFrame(columns=[season_col, 'stream', division_key_col, club_col,
                                     'n_players','avg_main_tier','median_main_tier'])

    # Each player's best (lowest number) Main tier this season
    main_tier_map = compute_player_main_tier_map(work, player_id_col, main_tier_col)

    # Merge and keep only those who actually played Main
    merged = (special
              .merge(main_tier_map, on=player_id_col, how='left')
              .dropna(subset=['player_main_tier']))

    # Dedupe to avoid double-counting same player within same (season, division, club)
    merged = merged.drop_duplicates(subset=[season_col, division_key_col, club_col, player_id_col])

    # Aggregate
    out = (merged
           .groupby([season_col, 'stream', division_key_col, club_col], as_index=False)
           .agg(n_players=(player_id_col, 'nunique'),
                avg_main_tier=('player_main_tier', 'mean'),
                median_main_tier=('player_main_tier', 'median')))

    # Tidy / sort
    def numeric_from_label(lbl: str):
        if isinstance(lbl, str):
            if 'premier' in lbl.lower():
                return 1
            m = re.search(r'(\d+)', lbl)
            if m:
                return int(m.group(1))
        return 999
    out['_sort_tier'] = out[division_key_col].apply(numeric_from_label)
    out = out.sort_values([season_col, 'stream', club_col, '_sort_tier', division_key_col]).drop(columns=['_sort_tier'])

    out['avg_main_tier'] = out['avg_main_tier'].round(2)
    out['median_main_tier'] = out['median_main_tier'].astype('Int64')

    return out

# Run for both seasons
avg_main_by_special_by_club_2425 = average_main_tier_for_special_divisions_by_club(players_2425)
avg_main_by_special_by_club_2526 = average_main_tier_for_special_divisions_by_club(players_2526)

display(avg_main_by_special_by_club_2425)
display(avg_main_by_special_by_club_2526)


,season,stream,division_key,Club,n_players,avg_main_tier,median_main_tier
7,2024-2025,Ladies,Premier Ladies,HKCC,3,2.33,2
0,2024-2025,Ladies,L2,HKCC,6,5.83,6
3,2024-2025,Ladies,L3,HKCC,5,8.6,7
5,2024-2025,Ladies,L4,HKCC,6,12.67,13
8,2024-2025,Ladies,Premier Ladies,HKFC,4,2.25,2
1,2024-2025,Ladies,L2,HKFC,14,6.29,5
4,2024-2025,Ladies,L3,HKFC,10,10.4,11
6,2024-2025,Ladies,L4,HKFC,10,12.7,13
9,2024-2025,Ladies,Premier Ladies,KCC,11,3.09,3
2,2024-2025,Ladies,L2,KCC,7,6.29,6


,season,stream,division_key,Club,n_players,avg_main_tier,median_main_tier
7,2025-2026,Ladies,Premier Ladies,HKCC,3,2.0,2
0,2025-2026,Ladies,L2,HKCC,6,5.5,5
3,2025-2026,Ladies,L3,HKCC,7,8.0,7
5,2025-2026,Ladies,L4,HKCC,8,12.88,13
8,2025-2026,Ladies,Premier Ladies,HKFC,2,2.0,2
1,2025-2026,Ladies,L2,HKFC,13,6.46,6
4,2025-2026,Ladies,L3,HKFC,11,9.45,11
6,2025-2026,Ladies,L4,HKFC,11,12.0,13
9,2025-2026,Ladies,Premier Ladies,KCC,9,2.89,2
2,2025-2026,Ladies,L2,KCC,11,5.18,6


In [32]:
avg_main_by_special_by_club_2425[avg_main_by_special_by_club_2425["division_key"] == "M3"]

,season,stream,division_key,Club,n_players,avg_main_tier,median_main_tier
12,2024-2025,Masters,M3,HKCC,12,10.33,11
13,2024-2025,Masters,M3,HKFC,17,6.59,6
14,2024-2025,Masters,M3,KCC,8,8.38,9


In [22]:
# ---------- Flexible column finders ----------
def find_player_id_col(df: pd.DataFrame):
    candidates = {'hks no.', 'hks no', 'hks_no', 'hksno', 'player_id', 'player no', 'player_no'}
    lower_map = {c.lower(): c for c in df.columns}
    for key in candidates:
        if key in lower_map:
            return lower_map[key]
    for c in df.columns:
        if 'hks' in c.lower():
            return c
    raise KeyError("Could not find an HKS/player ID column.")

def find_team_col(df: pd.DataFrame):
    candidates = {'team', 'team_name', 'home_team', 'away_team'}
    lower_map = {c.lower(): c for c in df.columns}
    for key in candidates:
        if key in lower_map:
            return lower_map[key]
    for c in df.columns:
        if 'team' in c.lower():
            return c
    raise KeyError("Could not find a Team column.")

def find_possible_name_cols(df: pd.DataFrame):
    # sweep a bunch of plausible variants
    keys = {'player','player name','player_name','name','full name','full_name'}
    out = []
    for c in df.columns:
        if c.lower() in keys or 'player' in c.lower() or 'name' in c.lower():
            out.append(c)
    return out

# ---------- Stream + division helpers ----------
def stream_from_division_key(x: str) -> str:
    if x is None: return 'Main'
    s = str(x).strip().lower()
    if 'ladies' in s or s.startswith('l'):   return 'Ladies'
    if 'masters' in s or s.startswith('m'):  return 'Masters'
    return 'Main'

def special_rank(div_key: str) -> float:
    if not isinstance(div_key, str): return np.inf
    s = div_key.strip().lower()
    if 'premier' in s: return 1
    m = re.search(r'(\d+)', s)
    return int(m.group(1)) if m else np.inf

def most_frequent(values):
    s = pd.Series(values).dropna()
    return None if s.empty else s.mode().iloc[0]

# ---------- Build a robust player-name lookup by HKS No. ----------
def build_player_name_lookup(df: pd.DataFrame, player_id_col: str) -> pd.DataFrame:
    name_cols = find_possible_name_cols(df)
    if not name_cols:
        # no name columns at all
        return pd.DataFrame({player_id_col: pd.Series(dtype=df[player_id_col].dtype),
                             'PlayerName': pd.Series(dtype='string')})
    # Melt across all candidate name cols to collect non-null names per HKS
    name_stack = []
    for c in name_cols:
        sub = df[[player_id_col, c]].dropna(subset=[c]).copy()
        sub.rename(columns={c: 'name_val'}, inplace=True)
        name_stack.append(sub)
    names = (pd.concat(name_stack, ignore_index=True)
               .dropna(subset=['name_val']))
    if names.empty:
        # names exist but all null
        return pd.DataFrame({player_id_col: df[player_id_col].drop_duplicates(), 'PlayerName': None})
    # Choose the most frequent spelling per HKS
    names['name_val'] = names['name_val'].astype(str).str.strip().replace({'': np.nan})
    names = names.dropna(subset=['name_val'])
    pick = (names.groupby(player_id_col)['name_val']
                 .agg(most_frequent)
                 .reset_index()
                 .rename(columns={'name_val': 'PlayerName'}))
    return pick

# ---------- Build per-player MAIN summary (best tier + team at that tier) ----------
def build_main_summary(df: pd.DataFrame, player_id_col: str, team_col: str) -> pd.DataFrame:
    # Require a valid Main tier
    main = df[(df['stream'] == 'Main') & (df['division_tier_main'].notna())].copy()
    if main.empty:
        return pd.DataFrame(columns=[player_id_col, 'player_main_tier', 'player_main_team'])
    best_tier = (main.groupby(player_id_col, as_index=False)['division_tier_main']
                      .min()
                      .rename(columns={'division_tier_main': 'player_main_tier'}))
    main = main.merge(best_tier, on=player_id_col, how='inner')
    at_best = main[main['division_tier_main'] == main['player_main_tier']]
    team_pick = (at_best.groupby(player_id_col, as_index=False)
                 .agg(player_main_team=(team_col, most_frequent)))
    out = best_tier.merge(team_pick, on=player_id_col, how='left')
    # Keep only players who actually have a main TEAM as well (non-null)
    out = out[out['player_main_team'].notna()]
    return out

# ---------- Build per-player Masters/Ladies summary (strongest division + team there) ----------
def build_special_summary(df: pd.DataFrame, stream: str, player_id_col: str, team_col: str) -> pd.DataFrame:
    sp = df[df['stream'] == stream].copy()
    if sp.empty:
        return pd.DataFrame(columns=[player_id_col, f'{stream}_division', f'{stream}_team', f'{stream}_rank'])
    sp['special_rank'] = sp['division_key'].apply(special_rank)
    best = (sp.groupby(player_id_col, as_index=False)['special_rank']
              .min()
              .rename(columns={'special_rank': f'{stream}_rank'}))
    sp = sp.merge(best, on=player_id_col, how='inner')
    sp_best = sp[sp['special_rank'] == sp[f'{stream}_rank']]
    agg = (sp_best.groupby(player_id_col, as_index=False)
           .agg(**{
               f'{stream}_division': ('division_key', most_frequent),
               f'{stream}_team': (team_col, most_frequent)
           }))
    # Keep only players who actually have a team label in that stream
    agg = agg[agg[f'{stream}_team'].notna()]
    # add rank (useful later)
    agg = agg.merge(best, on=player_id_col, how='left')
    return agg

# ================== Build 25/26 Masters & Ladies tables (ONLY players with BOTH teams) ==================
pid_col  = find_player_id_col(players_2526)
team_col = find_team_col(players_2526)

# Ensure stream column present
if 'stream' not in players_2526.columns:
    players_2526 = players_2526.copy()
    players_2526['stream'] = players_2526['division_key'].apply(stream_from_division_key)

p2526 = players_2526[players_2526['season'] == '2025-2026'].copy()

# Player name lookup from ANY rows in 25/26 (any stream)
name_map_2526 = build_player_name_lookup(p2526, pid_col)

# Build summaries
main_summary_2526    = build_main_summary(p2526, pid_col, team_col)
masters_summary_2526 = build_special_summary(p2526, 'Masters', pid_col, team_col)
ladies_summary_2526  = build_special_summary(p2526, 'Ladies',  pid_col, team_col)

# Require BOTH sides to exist: inner join with main
masters_players_2526 = masters_summary_2526.merge(main_summary_2526, on=pid_col, how='inner')
ladies_players_2526  = ladies_summary_2526.merge(main_summary_2526, on=pid_col, how='inner')

# Attach names (left join from the id→name map)
masters_players_2526 = masters_players_2526.merge(name_map_2526, on=pid_col, how='left')
ladies_players_2526  = ladies_players_2526.merge(name_map_2526, on=pid_col, how='left')

# Order columns neatly
def order_cols(df, stream: str):
    cols = [pid_col, 'PlayerName', f'{stream}_division', f'{stream}_team', 'player_main_tier', 'player_main_team']
    have = [c for c in cols if c in df.columns]
    return df[have]

masters_players_2526 = order_cols(masters_players_2526, 'Masters').sort_values(['player_main_tier', 'Masters_division', 'PlayerName'], na_position='last')
ladies_players_2526  = order_cols(ladies_players_2526,  'Ladies').sort_values(['player_main_tier', 'Ladies_division',  'PlayerName'], na_position='last')

display(masters_players_2526.head(20))
display(ladies_players_2526.head(20))
print(f"Masters (both teams present): {len(masters_players_2526)}")
print(f"Ladies  (both teams present): {len(ladies_players_2526)}")


,HKS No.,PlayerName,Masters_division,Masters_team,player_main_tier,player_main_team
150,32455,Gates Stuart,M2,Hong Kong Football Club M2B,1,Hong Kong Football Club 1B
18,19907,Wong Joe Wai Chung,Premier Masters,Hong Kong Cricket Club M1B,1,FWD
49,25295,Chiu Ho Fai,Premier Masters,Hong Kong Cricket Club M1A,2,Hong Kong Cricket Club D2
50,25313,Chiu Raymond Shung Hei,Premier Masters,Hong Kong Cricket Club M1A,2,Hong Kong Cricket Club D2
175,33627,Genever Peter John,Premier Masters,Hong Kong Cricket Club M1B,2,Hong Kong Cricket Club D2
31,22148,Hill Peter,Premier Masters,Hong Kong Cricket Club M1A,2,Hong Kong Cricket Club D2
105,29547,Ho Eric Chu Him,Premier Masters,Hong Kong Football Club M1A,2,Hong Kong Football Club 2A
23,21229,Khan Faheem,Premier Masters,Hong Kong Cricket Club M1B,2,Hong Kong Cricket Club D2
42,24673,Kwong Anson Yu Shun,Premier Masters,Hong Kong Cricket Club M1B,2,Kowloon Cricket Club 2
124,30509,Ward Andrew,Premier Masters,Hong Kong Football Club M1B,2,Hong Kong Football Club 2A


,HKS No.,PlayerName,Ladies_division,Ladies_team,player_main_tier,player_main_team
23,29632,Cheng Nga Ching,Premier Ladies,Kowloon Cricket Club L1A,2,Young Player 2
52,32309,Cheung Tsz Ching,Premier Ladies,Kowloon Cricket Club L1B,2,Young Player 2
32,30881,Fung Ching Hei,Premier Ladies,Hong Kong Football Club L1,2,Young Player 2
16,28466,Ho Tomato Tze Lok,Premier Ladies,Hong Kong Football Club L1,2,Hong Kong Football Club 2A
33,31022,Lam Po Ying,Premier Ladies,Kowloon Cricket Club L1A,2,Young Player 2
42,31882,Leung Ka Huen,Premier Ladies,JESSICA L1,2,Young Player 2
58,32618,Lo Pui Yin Chloe,Premier Ladies,Hong Kong Cricket Club L1,2,Young Player 2
7,27291,Tong Tsz Wing,Premier Ladies,Kowloon Cricket Club L1B,2,Kowloon Cricket Club 2
36,31163,Tse Toby Yee Lam,Premier Ladies,Kowloon Cricket Club L1A,2,Young Player 2
37,31176,Wai Sze Wing,Premier Ladies,Hong Kong Cricket Club L1,2,Hong Kong Cricket Club D2


Masters (both teams present): 210
Ladies  (both teams present): 121


In [23]:
masters_players_2526[masters_players_2526["Masters_division"] == "M4"]

,HKS No.,PlayerName,Masters_division,Masters_team,player_main_tier,player_main_team
22,20788,Au Siu Fai,M4,Hong Kong Football Club M4A,7,Hong Kong Football Club 7B
112,29926,Chan Dave Chi Keung,M4,Hong Kong Football Club M4B,7,Hong Kong Football Club 7B
46,25063,Chan George Yee Lee,M4,Hong Kong Football Club M4A,7,Hong Kong Football Club 7B
160,32906,Dansanghani Rahul,M4,Hong Kong Football Club M4B,7,Hong Kong Football Club 7A
131,30938,Dickson Robert,M4,Hong Kong Cricket Club M4,7,Hong Kong Cricket Club D7B
...,...,...,...,...,...,...
57,26010,Chiu Yee Lai,M4,SKCSC M4,14,North District Sports Association 2
137,31612,Ho Wai Keung,M4,Royal Hong Kong Yacht Club M4,14,Royal Hong Kong Yacht Club Div 14
165,33120,Lai Ming Wah,M4,Royal Hong Kong Yacht Club M4,14,Royal Hong Kong Yacht Club Div 14
157,32813,Chui Benjamin,M4,Royal Hong Kong Yacht Club M4,15,Royal Hong Kong Yacht Club Div 15


In [24]:
# Count the number of HKS No. that appear in both seasons
pid_col_2425 = find_player_id_col(players_2425)
pid_col_2526 = find_player_id_col(players_2526)
common_hks_nos = set(players_2425[pid_col_2425].dropna().unique()).intersection(set(players_2526[pid_col_2526].dropna().unique()))
print(f"Players appearing in BOTH seasons: {len(common_hks_nos)}")

Players appearing in BOTH seasons: 1406


In [27]:
# Count the number of HKS No. that appear in 2025/26 but not in 2024/25

pid_col_2425 = find_player_id_col(players_2425)
pid_col_2526 = find_player_id_col(players_2526)
hks_2425 = set(players_2425[pid_col_2425].dropna().unique())
hks_2526 = set(players_2526[pid_col_2526].dropna().unique())
new_hks_nos = hks_2526 - hks_2425
print(f"Players appearing in 2025/26 but NOT in 2024/25: {len(new_hks_nos)}")

Players appearing in 2025/26 but NOT in 2024/25: 327


In [28]:
1406/1715

0.8198250728862974

'HKS No.'

In [65]:
players_2425[players_2425["Team"].str.contains("Football")]["Team"].unique()

array(['Hong Kong Football Club 12A', 'Hong Kong Football Club 12B',
       'Hong Kong Football Club 14A', 'Hong Kong Football Club 14B',
       'Hong Kong Football Club 4A', 'Hong Kong Football Club 4B',
       'Hong Kong Football Club 7A', 'Hong Kong Football Club 7B',
       'Hong Kong Football Club 9', 'Hong Kong Football Club L4',
       'Hong Kong Football Club JSA', 'Hong Kong Football Club M3A',
       'Hong Kong Football Club M3B', 'Hong Kong Football Club M4A',
       'Hong Kong Football Club M4B', 'Hong Kong Football Club 2A',
       'Hong Kong Football Club 2B', 'Hong Kong Football Club L2A',
       'Hong Kong Football Club L2B', 'Hong Kong Football Club L2C',
       'Hong Kong Football Club L3A', 'Hong Kong Football Club L3B',
       'Hong Kong Football Club L3C', 'Hong Kong Football Club M2A',
       'Hong Kong Football Club M2B', 'Hong Kong Football Club M1A',
       'Hong Kong Football Club M1B', 'Hong Kong Football Club 3A',
       'Hong Kong Football Club 3B', 'Hong K

In [67]:
def find_player_id_col(df: pd.DataFrame):
    candidates = {'hks no.', 'hks no', 'hks_no', 'hksno', 'player_id', 'player no', 'player_no'}
    lower_map = {c.lower(): c for c in df.columns}
    for key in candidates:
        if key in lower_map: return lower_map[key]
    for c in df.columns:
        if 'hks' in c.lower(): return c
    raise KeyError("Could not find an HKS/player ID column.")

def find_team_col(df: pd.DataFrame):
    candidates = {'team', 'team_name', 'home_team', 'away_team'}
    lower_map = {c.lower(): c for c in df.columns}
    for key in candidates:
        if key in lower_map: return lower_map[key]
    for c in df.columns:
        if 'team' in c.lower(): return c
    raise KeyError("Could not find a Team column.")

def find_possible_name_cols(df: pd.DataFrame):
    # Collect any plausible player-name columns
    out = []
    for c in df.columns:
        cl = c.lower()
        if cl in {'player','player name','player_name','name','full name','full_name'} or 'player' in cl or 'name' in cl:
            out.append(c)
    return out

def most_frequent(values):
    s = pd.Series(values).dropna()
    return None if s.empty else s.mode().iloc[0]

def build_player_name_lookup(df: pd.DataFrame, player_id_col: str) -> pd.DataFrame:
    name_cols = find_possible_name_cols(df)
    if not name_cols:
        return pd.DataFrame({player_id_col: df[player_id_col].drop_duplicates(), 'Player': None})
    stacks = []
    for c in name_cols:
        sub = df[[player_id_col, c]].dropna(subset=[c]).copy()
        sub.rename(columns={c: 'name_val'}, inplace=True)
        stacks.append(sub)
    if not stacks:
        return pd.DataFrame({player_id_col: df[player_id_col].drop_duplicates(), 'Player': None})
    names = (pd.concat(stacks, ignore_index=True)
               .assign(name_val=lambda x: x['name_val'].astype(str).str.strip())
               .replace({'name_val': {'': np.nan}})
               .dropna(subset=['name_val']))
    pick = (names.groupby(player_id_col)['name_val'].agg(most_frequent)
                 .reset_index()
                 .rename(columns={'name_val': 'Player'}))
    return pick

def build_main_summary(df: pd.DataFrame, player_id_col: str, team_col: str) -> pd.DataFrame:
    """
    Return one row per player present in Main: best (lowest) division_tier_main and a representative Main team at that tier.
    """
    main = df[df['division_tier_main'].notna()].copy()  # already Main-only due to how division_tier_main is defined
    if main.empty:
        return pd.DataFrame(columns=[player_id_col, 'division_tier_main', 'Team'])
    # Best tier per player
    best_tier = (main.groupby(player_id_col, as_index=False)['division_tier_main']
                      .min()
                      .rename(columns={'division_tier_main': 'division_tier_main'}))
    joined = main.merge(best_tier, on=player_id_col, how='inner', suffixes=('', '_best'))
    at_best = joined[joined['division_tier_main'] == joined['division_tier_main_best']]
    team_pick = (at_best.groupby(player_id_col, as_index=False)
                 .agg(Team=(team_col, most_frequent)))
    out = best_tier.drop(columns=['division_tier_main_best'], errors='ignore').merge(team_pick, on=player_id_col, how='left')
    # keep only rows with a real team label
    out = out[out['Team'].notna()]
    return out


In [68]:
# Identify columns
pid_2425 = find_player_id_col(players_2425)
pid_2526 = find_player_id_col(players_2526)
assert pid_2425 == pid_2526, "HKS No. column differs between seasons."
pid_col = pid_2425

team_2425 = find_team_col(players_2425)
team_2526 = find_team_col(players_2526)

# Optional: keep only the correct season rows (in case frames contain more)
p2425 = players_2425[players_2425['season'] == '2024-2025'].copy()
p2526 = players_2526[players_2526['season'] == '2025-2026'].copy()

# Build main summaries
main_2425 = build_main_summary(p2425, pid_col, team_2425).rename(
    columns={'division_tier_main': 'division_tier_main_2024_25', 'Team': 'Team_2024_25'}
)
main_2526 = build_main_summary(p2526, pid_col, team_2526).rename(
    columns={'division_tier_main': 'division_tier_main_2025_26', 'Team': 'Team_2025_26'}
)

# Build name lookups (prefer latest season name if available)
names_2425 = build_player_name_lookup(p2425, pid_col)
names_2526 = build_player_name_lookup(p2526, pid_col)

# Merge both seasons on HKS No. (inner: players who have Main in BOTH seasons)
compare_main = (main_2425.merge(main_2526, on=pid_col, how='inner')
                         .merge(names_2526, on=pid_col, how='left')
                         .merge(names_2425.rename(columns={'Player':'Player_2425'}), on=pid_col, how='left'))

# Prefer 25/26 name, else fall back to 24/25
compare_main['Player'] = compare_main['Player'].fillna(compare_main['Player_2425'])
compare_main = compare_main.drop(columns=['Player_2425'])

# Compute change (positive = moved to a higher-numbered tier; negative = moved up)
compare_main['delta_tier'] = (compare_main['division_tier_main_2025_26']
                              - compare_main['division_tier_main_2024_25'])

# Order columns and sort
cols = ['Player', pid_col, 'division_tier_main_2024_25', 'Team_2024_25',
        'division_tier_main_2025_26', 'Team_2025_26', 'delta_tier']
compare_main = compare_main[cols].sort_values(['delta_tier','division_tier_main_2025_26','Player'], na_position='last')

display(compare_main.head(30))
print(f"Players with Main in BOTH seasons: {len(compare_main)}")


,Player,HKS No.,division_tier_main_2024_25,Team_2024_25,division_tier_main_2025_26,Team_2025_26,delta_tier
1218,Chung Shun Yeung,35022,13,i-Mask Advance Squash Club 5,4,Sha Tin 1,-9
974,Kwok Yau Yau,34169,13,TNG 1,5,Sha Tin 2,-8
841,Choe Yeda,33701,14,Ladies Recreation Club Div 14,6,Ladies Recreation Club Div 6,-8
909,Verma Kanay,33958,15,HKIS Div 15,8,Fusion Squash Club 2,-7
1055,Wong Tsz Wang Clayton,34413,15,Squashathon One,8,Squashathon Zero,-7
800,Chung King Long Jacqueline,33463,13,TNG 2,7,Hong Kong Cricket Club D7B,-6
1017,Zhao Ling Ban,34292,13,Energy Squash Club 1,7,Kinetic 1,-6
893,Lin Zixuan Emma,33884,14,8 Virtues + 1,8,TNG 1,-6
446,Law Hong Ting Kevin,30201,11,Global Squash 2,6,Global Squash 1,-5
951,Bai Yuet,34086,13,TNG 1,8,X-Alpha 4,-5


Players with Main in BOTH seasons: 1246


In [69]:
compare_main.tail(30)

,Player,HKS No.,division_tier_main_2024_25,Team_2024_25,division_tier_main_2025_26,Team_2025_26,delta_tier
833,Mercalfe Stephen Fraser,33662,7,Hong Kong Cricket Club 5,10,Hong Kong Cricket Club D10,3
957,Warner James William,34111,7,Hong Kong Cricket Club 6,10,Hong Kong Cricket Club D10,3
600,Chan Ying Lun,31866,8,Squashathon Zero,11,The Zero Squash Club D11,3
160,Wong Tony Wing Tung,26395,9,Friend Club,12,SKCSC Div 12,3
681,Chan Simon,32548,10,Craigengower Cricket Club,13,Craigengower Cricket Club,3
99,Chow Pui Sing,23591,10,Bravo D10,13,Bravo D13,3
757,Chui Horace,33180,10,Craigengower Cricket Club,13,Craigengower Cricket Club,3
136,Lam Wilson Koon Fung,25503,10,Craigengower Cricket Club,13,Craigengower Cricket Club,3
810,Li Chung Him Samuel,33518,10,Craigengower Cricket Club,13,Craigengower Cricket Club,3
978,Lo Tien Yue,34181,10,Craigengower Cricket Club,13,Craigengower Cricket Club,3


In [71]:
compare_main[compare_main["Team_2025_26"].str.contains("Kong Cricket")]

,Player,HKS No.,division_tier_main_2024_25,Team_2024_25,division_tier_main_2025_26,Team_2025_26,delta_tier
800,Chung King Long Jacqueline,33463,13,TNG 2,7,Hong Kong Cricket Club D7B,-6
1202,Poole-Wilson Michael,34983,11,HKCC Tuesday Night Rockers,7,Hong Kong Cricket Club D7A,-4
843,Leung Ka Hei,33708,7,Hong Kong Cricket Club 5,4,Hong Kong Cricket Club D4B,-3
809,Achouch Jules Simon,33513,6,Hong Kong Cricket Club 4,4,Hong Kong Cricket Club D4B,-2
761,Sheethal Kumar Palash,33207,6,Hong Kong Cricket Club 4,4,Hong Kong Cricket Club D4B,-2
...,...,...,...,...,...,...,...
484,Bruwer Frank,30507,4,Hong Kong Cricket Club 3,6,Hong Kong Cricket Club D6,2
531,Dalpathraj Sheethal Kumar,31120,7,Hong Kong Cricket Club 5,10,Hong Kong Cricket Club D10,3
126,Lai Kwok Wai Wilfred,25178,7,Hong Kong Cricket Club 6,10,Hong Kong Cricket Club D10,3
833,Mercalfe Stephen Fraser,33662,7,Hong Kong Cricket Club 5,10,Hong Kong Cricket Club D10,3


In [72]:
# --- helpers reused ---
def find_player_id_col(df: pd.DataFrame):
    for key in ['HKS No.', 'HKS No', 'HKS_No', 'HKSNO', 'player_id', 'Player No', 'Player_No']:
        if key in df.columns: return key
    for c in df.columns:
        if 'hks' in c.lower(): return c
    raise KeyError("HKS No. column not found")

def find_team_col(df: pd.DataFrame):
    # prefer 'Team' but be tolerant
    preferred = ['Team','team','Team_Name','team_name','Home Team','home_team']
    for key in preferred:
        if key in df.columns: return key
    for c in df.columns:
        if 'team' in c.lower(): return c
    raise KeyError("Team column not found")

def compute_player_main_tier_map(df: pd.DataFrame, player_id_col: str, main_tier_col='division_tier_main') -> pd.DataFrame:
    main = df[df[main_tier_col].notna()].copy()
    if main.empty:
        return pd.DataFrame(columns=[player_id_col, 'player_main_tier'])
    best = (main.groupby(player_id_col, as_index=False)[main_tier_col]
                .min()
                .rename(columns={main_tier_col: 'player_main_tier'}))
    return best

def is_L2(lbl: str) -> bool:
    if not isinstance(lbl, str): return False
    s = re.sub(r'\s+', '', lbl).upper()
    return s == 'L2'

# --- filter to 25/26 and build maps ---
pid_col  = find_player_id_col(players_2526)
team_col = find_team_col(players_2526)

p2526 = players_2526[players_2526['season'] == '2025-2026'].copy()

# players registered/appearing in Ladies L2
l2_rows = p2526[p2526['division_key'].apply(is_L2)].copy()

# get each player's best (lowest) Main tier in 25/26
main_map_2526 = compute_player_main_tier_map(p2526, pid_col, 'division_tier_main')

# attach main tier to L2 players; drop those with no Main appearances
l2_with_main = (l2_rows[[pid_col, team_col]]
                .drop_duplicates()
                .merge(main_map_2526, on=pid_col, how='left')
                .dropna(subset=['player_main_tier']))

# --- pivot: counts of players by Team × Main tier ---
pivot_counts = (l2_with_main
    .assign(player_main_tier=lambda d: d['player_main_tier'].astype('Int64'))
    .pivot_table(index=team_col,
                 columns='player_main_tier',
                 values=pid_col,
                 aggfunc='nunique',
                 fill_value=0)
    .sort_index())

# --- per-team summary stats (n, avg, median) ---
summary = (l2_with_main
    .groupby(team_col, as_index=False)
    .agg(n_players=(pid_col, 'nunique'),
         avg_main_tier=('player_main_tier', 'mean'),
         median_main_tier=('player_main_tier', 'median')))

summary['avg_main_tier'] = summary['avg_main_tier'].round(2)
summary['median_main_tier'] = summary['median_main_tier'].astype('Int64')

display(pivot_counts)   # distribution table (Team x Main tier)
display(summary.sort_values(['avg_main_tier', 'median_main_tier', team_col]))


player_main_tier,3,4,5,6,7,9,12
Team,,,,,,,
Hong Kong Cricket Club L2,0,1,2,2,1,0,0
Hong Kong Football Club L2A,0,0,4,1,1,1,1
Hong Kong Football Club L2B,0,0,2,1,2,0,0
JESSICA L2,0,1,1,1,0,0,0
Kowloon Cricket Club L2A,1,1,2,2,0,0,0
Kowloon Cricket Club L2B,0,1,0,4,0,0,0


,Team,n_players,avg_main_tier,median_main_tier
4,Kowloon Cricket Club L2A,6,4.83,5
3,JESSICA L2,3,5.0,5
0,Hong Kong Cricket Club L2,6,5.5,5
5,Kowloon Cricket Club L2B,5,5.6,6
2,Hong Kong Football Club L2B,5,6.0,6
1,Hong Kong Football Club L2A,8,6.75,5
